## Video Referrence
- [Complete Tutorial on Vector Database - Learn ChromaDB, Pinecone & Weaviate | Generative AI](https://www.youtube.com/watch?v=8KrTO9bS91s)
- https://github.com/entbappy/Complete-Generative-AI-Course-on-YouTube/blob/main/Vector%20Database/2.Pinecone_demo.ipynb


## Import All the Required Libraries

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader, PyPDFLoader
from langchain.document_loaders import PyMuPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [ ]:
# Load API keys from .env file
# from dotenv import load_dotenv, find_dotenv
# load_dotenv(find_dotenv())

True

## Extract the Text from the PDF's

In [91]:
def load_pdfs_from_folder(folder_path: str) -> list[Document]:
    all_docs: list[Document] = []
    for filename in os.listdir(folder_path):
        if not filename.lower().endswith(".pdf"):
            continue

        full_path = os.path.join(folder_path, filename)
        loader = PyMuPDFLoader(full_path)
        docs = loader.load()

        # Derive title from filename, e.g. "My Paper.pdf" -> "My Paper"
        title = os.path.splitext(filename)[0]

        # Add the title meta-field to each Document
        for doc in docs:
            # doc.metadata already contains things like 'source'; we just add 'Title'
            doc.metadata["Title"] = title

        all_docs.extend(docs)

    return all_docs


# ✅ Load all PDFs in the folder
data = load_pdfs_from_folder("website_content/")

In [ ]:
# loader = PyPDFDirectoryLoader("website_content")
# # loader = PyPDFLoader("Website_Report_V1.pdf")
# data = loader.load()

Ignoring wrong pointing object 240 0 (offset 0)
Ignoring wrong pointing object 74 0 (offset 0)
Ignoring wrong pointing object 69 0 (offset 0)


In [92]:
data

[Document(metadata={'producer': 'Skia/PDF m138 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'website_content/Diesel-Dilemma-Page.pdf', 'file_path': 'website_content/Diesel-Dilemma-Page.pdf', 'total_pages': 8, 'format': 'PDF 1.4', 'title': 'Diesel-Dilemma-Page', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0, 'Title': 'Diesel-Dilemma-Page'}, page_content='Diesel Dilemma Page Analysis  \nEscalation of Subsidy Expenditure\u200b\nBetween 2019 and 2023, Malaysia’s diesel-subsidy outlay soared from RM 1.4 billion to RM \n14.3 billion—a more than ten-fold increase (Ministry of Finance Malaysia, 2024). This 920 \npercent rise placed unsustainable pressure on public finances, consuming an ever-larger \nshare of the federal budget. According to International Monetary Fund guidelines, untargeted \nfuel subsidies tend to crowd out essential social and capital spending, undermining \nlong-term growth pro

## Clean Data

In [93]:
import re

def clean_page(text: str) -> str:
    # Remove artificial newlines that break up sentences or words
    text = re.sub(r'\n+', ' ', text)             # Merge multiple newlines into one space
    text = re.sub(r'(?<=\w)-\s+(?=\w)', '', text) # Fix hyphenated line breaks (e.g., "subsi-\ndy" → "subsidy")
    text = re.sub(r'\s+', ' ', text)             # Normalize extra spaces
    return text.strip()

def clean_documents(docs: list[Document]) -> list[Document]:
    cleaned_docs = []
    for doc in docs:
        cleaned_text = clean_page(doc.page_content)
        cleaned_docs.append(Document(page_content=cleaned_text, metadata=doc.metadata))
    return cleaned_docs


In [94]:
clean_data = clean_documents(data)
print(clean_data)

[Document(metadata={'producer': 'Skia/PDF m138 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'website_content/Diesel-Dilemma-Page.pdf', 'file_path': 'website_content/Diesel-Dilemma-Page.pdf', 'total_pages': 8, 'format': 'PDF 1.4', 'title': 'Diesel-Dilemma-Page', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0, 'Title': 'Diesel-Dilemma-Page'}, page_content='Diesel Dilemma Page Analysis Escalation of Subsidy Expenditure\u200b Between 2019 and 2023, Malaysia’s diesel-subsidy outlay soared from RM 1.4 billion to RM 14.3 billion—a more than ten-fold increase (Ministry of Finance Malaysia, 2024). This 920 percent rise placed unsustainable pressure on public finances, consuming an ever-larger share of the federal budget. According to International Monetary Fund guidelines, untargeted fuel subsidies tend to crowd out essential social and capital spending, undermining long-term growth prospects (IMF, 2

In [95]:
print(clean_data[0])

page_content='Diesel Dilemma Page Analysis Escalation of Subsidy Expenditure​ Between 2019 and 2023, Malaysia’s diesel-subsidy outlay soared from RM 1.4 billion to RM 14.3 billion—a more than ten-fold increase (Ministry of Finance Malaysia, 2024). This 920 percent rise placed unsustainable pressure on public finances, consuming an ever-larger share of the federal budget. According to International Monetary Fund guidelines, untargeted fuel subsidies tend to crowd out essential social and capital spending, undermining long-term growth prospects (IMF, 2019). By highlighting this dramatic fiscal escalation, the policy page rightly frames the reform as a necessary corrective to restore budgetary balance and fiscal space. Disparity Between Fuel Consumption and Vehicle Growth​ The dual charts contrasting cumulative diesel consumption against vehicle-registration growth (2021–2023) reveal a widening gap: while the registered diesel-vehicle fleet grew modestly, total fuel consumption climbed st

## Split the Extracted Data into Text Chunks

In [96]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(clean_data)

In [98]:
print(len(text_chunks))
text_chunks[0]

96


Document(metadata={'producer': 'Skia/PDF m138 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': 'website_content/Diesel-Dilemma-Page.pdf', 'file_path': 'website_content/Diesel-Dilemma-Page.pdf', 'total_pages': 8, 'format': 'PDF 1.4', 'title': 'Diesel-Dilemma-Page', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0, 'Title': 'Diesel-Dilemma-Page'}, page_content='Diesel Dilemma Page Analysis Escalation of Subsidy Expenditure\u200b Between 2019 and 2023, Malaysia’s diesel-subsidy outlay soared from RM 1.4 billion to RM 14.3 billion—a more than ten-fold increase (Ministry of Finance Malaysia, 2024). This 920 percent rise placed unsustainable pressure on public finances, consuming an ever-larger share of the federal budget. According to International Monetary Fund guidelines, untargeted fuel subsidies tend to crowd out essential social and capital spending,')

## Load OPENAI API

In [ ]:
os.environ['OPENAI_API_KEY'] = 

## Download the Embeddings

In [100]:
# Use a specific embedding model
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
text = "LangChain is an AI framework for LLMs."
vector = embeddings_model.embed_query(text)

print(len(vector))
print(vector[:5])  # Print first 5 values for readability


1536
[0.0009549352689646184, 0.0018590498948469758, 0.0324464850127697, -0.013298600912094116, 0.039292510598897934]


## Initializing the Pinecone

In [101]:
import pinecone

os.environ['PINECONE_API_KEY'] = "pcsk_7HmYTn_KS4n9fp4CzxTjTrKpYWaaBgHvP2JPRRx9fp5URALDkKuCC1yeZYhbZ557rEfjYT"
pc = pinecone.Pinecone(os.getenv('PINECONE_API_KEY'))
index_name = "fit5120-tm01"
index = pc.Index(index_name)

## Create Embeddings for each of the Text Chunk

In [102]:
# Pinecone.from_texts() vs Pinecone.from_documents()
# .from_documents() stores meta data while .from_texts() does not

docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings_model, index_name=index_name)

## If you already have an index, you can load it like this

In [20]:
docsearch = Pinecone.from_existing_index(index_name, embeddings_model)
docsearch

## Similarity Search

In [23]:
query = "How much is the diesel subisdy expenditure in 2024"

docs = docsearch.similarity_search(query, k=3)

In [24]:
docs

[Document(metadata={}, page_content='understanding.\n \n \nDiesel  Dilemma  Page  Analysis   \nEscalation  of  Subsidy  Expenditure  \nBetween\n \n2019\n \nand\n \n2023,\n \nMalaysia’s\n \ndiesel-subsidy\n \noutlay\n \nsoared\n \nfrom\n \nRM\n \n1.4\n \nbillion\n \nto\n \nRM\n \n14.3\n \nbillion—a\n \nmore\n \nthan\n \nten-fold\n \nincrease\n \n(Ministry\n \nof\n \nFinance\n \nMalaysia,\n \n2024).\n \nThis\n \n920\n \npercent\n \nrise\n \nplaced\n \nunsustainable\n \npressure\n \non\n \npublic\n \nfinances,\n \nconsuming\n \nan\n \never-larger\n \nshare\n \nof\n \nthe\n \nfederal\n \nbudget.\n \nAccording\n \nto'),
 Document(metadata={}, page_content='4.  Post-Reform  Period  (Q2  2024–Q2  2025)  \n●  No  Step  Increase:  Unlike  Peninsular  diesel,  the  Diesel  East  series  shows  no  \nsignificant\n \njump\n \nin\n \nQ2\n \n2024.\n \nIt\n \nremains\n \nat\n \n~RM\n \n2.18/L\n \nthroughout\n \n2024–2025.\n \n ●  Implication:  The  targeted-subsidy  reform  of  June  2024  did  not  

## Creating a LLM Model Wrapper

In [25]:
llm_gpt4 = ChatOpenAI(model="gpt-4.1-nano") #gpt-4.1-mini

qa = RetrievalQA.from_chain_type(llm=llm_gpt4, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={"k": 3}))

## Q/A

In [26]:
query = "How much is the diesel subisdy expenditure in 2024"
qa.invoke(query)

{'query': 'How much is the diesel subisdy expenditure in 2024',
 'result': 'The diesel subsidy expenditure in Malaysia in 2024 is not explicitly provided in the given data. However, it is stated that the expenditure soared from RM 1.4 billion in 2019 to RM 14.3 billion in 2023, representing a more than ten-fold increase. Since 2024 falls after 2023, the expenditure is likely still very high, but the exact figure for 2024 is not specified.'}

In [27]:
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    break
  if user_input == '':
    continue
  result = qa.invoke({'query': user_input})
  print(f"Answer: {result['result']}")
     

Answer: The diesel subsidy expenditure in Malaysia in 2023 was RM 14.3 billion.
Exiting
